In [4]:
import dreamer as dm
import numpy as np
import sys
import os
import torch

import gymnasium as gym
from importlib import reload
import time

sys.path.append(os.path.abspath('../hockey_env/hockey'))
import hockey_env as h_env


In [8]:
(torch.rand(4) - 0.5) * 2

tensor([-0.1926,  0.7666,  0.9174,  0.8494])

In [4]:
env = h_env.HockeyEnv()

print(env.observation_space.shape[0])
print(env.action_space.shape[0])

18
8


In [6]:
d = dm.DreamerV3(
	env=env,
    obs_dim=env.observation_space.shape[0],
    action_dim=env.action_space.shape[0],
    latent_dim=8,
    latent_categories_size=32,
    model_dim=256,
    imagination_horizon=15)

latent_size:  8
action_size:  8


In [8]:
d.generate_trajectories(1,1)

TypeError: expected Tensor as element 0 in argument 0, but got list